In [13]:
%cd ..
%rm -rf no-time-to-backprop

/content


In [14]:
!ls
!nvidia-smi

sample_data
Wed Dec 17 18:15:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   65C    P0             29W /   70W |     110MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------

In [15]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.80"

!git clone https://github.com/Francis2002/no-time-to-backprop.git
%cd no-time-to-backprop

Cloning into 'no-time-to-backprop'...
remote: Enumerating objects: 192, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 192 (delta 32), reused 32 (delta 14), pack-reused 138 (from 1)
Receiving objects: 100% (192/192), 88.26 MiB | 23.83 MiB/s, done.
Resolving deltas: 100% (101/101), done.
/content/no-time-to-backprop


In [16]:
!python -V
!ls

Python 3.12.12
constraints.txt			 requirements-macos-cpu.txt  run_on_gpu.ipynb
hpt.py				 requirements.mac.txt	     run_toy_task.py
plotting_scripts		 requirements.txt	     test_rec.py
requirements-base.txt		 run_all_benchmarks.sh	     tgap
requirements.from_linux_env.txt  run_all.sh
requirements-linux-cuda.txt	 run_all_toy.sh


In [4]:
%pip install -r requirements-linux-cuda.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.0/101.0 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.2/488.2 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 98.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.1/309.1 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 150.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 125.5 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

In [17]:
import jax
import jaxlib
import sys

print(f"jax: {jax.__version__}")
print(f"jaxlib: {jaxlib.__version__}")
print(f"executable: {sys.executable}")
print(f"devices: {jax.devices()}")


jax: 0.8.1
jaxlib: 0.8.1
executable: /usr/bin/python3
devices: [CudaDevice(id=0)]


In [ ]:
!python3 run_toy_task.py -m ONLINE -a ZUC --dataset mooc --task link_classification

In [ ]:
# Here we run with new sampler
!python3 run_toy_task.py -m ONLINE -a ZUC --num_layers 4 --num_hidden 128 --activation full_glu --decoder MLP --mixing rotational_full --prenorm --encoder --layer_output --extra_skip --dataset mooc --task link_classification --batching_strategy rearranged --lr 0.001 --dropout 0.15 --weight_decay 0.005 --batch_size 50 --num_epochs 200 --num_gradient_accumulation_steps 32 --dedupe --lr_schedule warmup_cosine

In [ ]:
# Here we ran with old sampler to compare if they are the same under batching strategy none
!python3 run_toy_task.py -m ONLINE -a ZUC --num_layers 4 --num_hidden 128 --activation full_glu --decoder MLP --mixing rotational_full --prenorm --encoder --layer_output --extra_skip --dataset mooc --task link_classification --batching_strategy none --lr 0.001 --dropout 0.15 --weight_decay 0.005 --batch_size 50 --num_epochs 200 --num_gradient_accumulation_steps 32 --dedupe --lr_schedule warmup_cosine

[*] Results will be stored in /content/no-time-to-backprop/results_for_rotational_vs_none/mooc_ONLINE
[*] d_model set to 64
Running iteration: 0
[*] Output file /content/no-time-to-backprop/tgap/data/npzs/mooc_stream.npz already exists. Using existing file.
W1217 18:15:31.434465    1885 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.
W1217 18:15:31.436585    1870 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.
[*] Created stream sampler with 288224 events, 7047 nodes (total regardless of splits), feat dim 6, target dim 1
Number of positive labels: 2977
[*] Created stream sampler with 61762 events, 7047 nodes (total regardless of splits), feat dim 6, target dim 1
Number of positive labels: 510
[*] Created stream sampler with 61763 events, 7047 nodes (total regardless of 

In [ ]:
!python run_toy_task.py -a ZUC -m ONLINE --num_epochs 5000 --num_steps 1000 --num_nodes 100 --num_layers 2 --num_hidden 32 --memory 2 --activation full_glu --prenorm --batch_size 50 --encoder --layer_output --extra_skip --decoder MLP --mixing rotational_full --acc --dont_store_results --steps_for_scheduler 20 --lr_schedule cosine --rec_lr_schedule cosine --use_plateau --plateau_factor 0.2 --plateau_patience 20 --lr_min 1e-6 --rec_learning_rate 0.001

[*] Results will be stored in /content/no-time-to-backprop/results_for_rotational_vs_none/toy_ONLINE
[*] NOT STORING RESULTS!!! to disk as per user request.
[*] d_model set to 16
[*] Forcing task to be link_regression, because dataset does not support classification
Running iteration: 0
[*] Created toy data with 100 nodes
W1215 13:53:10.118872    8385 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.
W1215 13:53:10.120944    8364 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.
[*] Trainable Parameters: 10129
[*] Model running on device: cuda:0
All params tree leaf keys:
|- cell
|  |- params
|  |  |- encoder
|  |  |  |- encoder
|  |  |  |  |- kernel  shape=(1, 16)
|  |  |  |  |- bias  shape=(16,)
|  |  |  |- layers_0
|  |  |  |  |- norm
|  |  |  |  |  |- scale  shape=(16,)